In [1]:
import numpy as np
import uproot
import h5py
import os
import torch
import torch_geometric.nn as PyG
from torch_geometric.transforms import Distance
from torch_geometric.data import InMemoryDataset as PyGDataset, Data as PyGData
from torch_geometric.data import Data
import math
import numba
import numpy as np
import torch
import h5py
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as PyG
from torch_geometric.transforms import Distance
from torch_geometric.data import DataLoader
from torch_geometric.data import Data as PyGData
from torch_geometric.data import Data
import sys, os
import subprocess
import csv, yaml
import math
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import torch.optim as optim
import argparse
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.tri as tri
# import numpy, numba, awkward, awkward.numba

In [2]:

# path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex2/220323_084833/delphes'
# graph_path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex2/220323_084833/graph/'
# output_path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex2/220323_084833/h5/'


# path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex1/220324_055935/delphes'
# graph_path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex1/220324_055935/graph/'
# output_path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex1/220324_055935/h5_2/'

## 4top
# # path = '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2/'
# path = '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520/'
# # path = '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_0/'

# # out_path1 = '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_w1/'
# out_path1 = '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w1/'
# # out_path1 = '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_0_w1/'

# # out_path2 = '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_w2/'
# out_path2 = '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_w2/'
# # out_path2 = '/users/yewzzang/data_graph/pt/4top_weight_210709/ex1_v2_210520_0_w2/'



### ttbar
path = '/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge/'
# out_path1 = '/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/'
out_path2 = '/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w2/'

In [3]:
# resampling_model_9s = '/users/yewzzang/work/HEPGNN_4top_resampling/result/date20220713_resampling_fea4_9s_ttbar_test1_alledge'
resampling_model_all = '/users/yewzzang/work/HEPGNN_4top_resampling/result/date20220713_resampling_fea4_all_ttbar_test1_alledge'

In [4]:
sys.path.append("./python")

In [5]:
from model.allModel import *

# model_9s = torch.load(resampling_model_9s+'/model.pth', map_location='cpu')
# model_9s.load_state_dict(torch.load(resampling_model_9s+'/weight.pth', map_location='cpu'))
# model_9s = model_9s.cuda()
# device = 'cuda'

model_all = torch.load(resampling_model_all+'/model.pth', map_location='cpu')
model_all.load_state_dict(torch.load(resampling_model_all+'/weight.pth', map_location='cpu'))
model_all = model_all.cuda()
device = 'cuda'

In [6]:
res = []
for root, dirs, files in os.walk(path):
    rootpath = os.path.join(os.path.abspath(path), root)
    for file in files:
        filepath = os.path.join(rootpath, file)
        
        filetype = filepath.split('.')[-1]
        if filetype != 'pt': continue
  
        res.append(filepath)

In [7]:
res

['/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge/ttbar_2746_0.pt',
 '/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge/ttbar_1803_0.pt',
 '/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge/ttbar_5825_0.pt',
 '/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge/ttbar_2811_0.pt',
 '/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge/ttbar_5178_0.pt',
 '/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge/ttbar_4327_0.pt',
 '/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge/ttbar_4394_0.pt',
 '/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge/ttbar_3028_0.pt',
 '/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge/ttbar_3690_0.pt',
 '/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge/ttbar_4514_0.pt',
 '/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge/ttbar_5379_0.pt',
 '/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge/ttbar_6720_0.pt',
 '/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge/ttba

In [8]:
import time
start = time.time()
for i in range(len(res)):

    file_name = res[i]
    cla = file_name.split('/')[-4].split('_')
    num = file_name.split('/')[-1].split('.')[-2]

    

    save_f_name = num

    save_path1 = out_path1
    save_path2 = out_path2
#     if not os.path.exists(save_path1): os.makedirs(save_path1)
    if not os.path.exists(save_path2): os.makedirs(save_path2)
#     save_file_name1 = save_path1 + save_f_name + '.pt'
    save_file_name2 = save_path2 + save_f_name + '.pt'
#     print(save_file_name)
### load data
    try:
        r_data = torch.load(file_name)
#     r_data = uproot.open(file_name)["Delphes"]
    except KeyError:
        print("delphes 없음")
        pass
    

    model_9s.eval()
    model_all.eval()
    
    
    list_9s = []
    list_all = []
    for i in range(len(r_data)):
#     for i in range(1):
        data = r_data[i].to(device)
        data_1 = PyGData(x = data.x[:,:4], edge_index=data.edge_index, pos = data.pos, y = data.y, batch = torch.from_numpy(np.zeros([1],dtype = np.int)))
        data_1 = data_1.to(device)
        
#         pred1 = model_9s(data_1)
        pred2 = model_all(data_1)
        
#         data_9s = data
        data_all = data
        

#         data_9s.rs = pred1[0].detach().to("cpu")
#         data_9s.x = data.x.detach().to("cpu")
#         data_9s.edge_index = data.edge_index.detach().to("cpu")
#         data_9s.pos = data.pos.detach().to("cpu")
#         data_9s.y = data.y.detach().to("cpu")
        
        data_all.rs = pred2[0].detach().to("cpu")
        data_all.x = data.x.detach().to("cpu")
        data_all.edge_index = data.edge_index.detach().to("cpu")
        data_all.pos = data.pos.detach().to("cpu")
        data_all.y = data.y.detach().to("cpu")
        
#         list_9s.append(data_9s)
        list_all.append(data_all)
        
#     torch.save(list_9s, save_file_name1)
    torch.save(list_all, save_file_name2)
    
#     print(save_file_name1,'finish')
    print(save_file_name1,'finish')
    
print(time.time()-start)

/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_2746_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_2746_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1803_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1803_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5825_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5825_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_2811_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_2811_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5178_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5178_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4327_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4327_0.pt

/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_6905_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_6905_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_3008_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_3008_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_3079_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_3079_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4663_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4663_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5258_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5258_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5855_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5855_0.pt

/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5518_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5518_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1860_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1860_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1987_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1987_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_3919_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_3919_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1448_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1448_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5824_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5824_0.pt

/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_6743_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_6743_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4673_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4673_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_595_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_595_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4750_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4750_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_3163_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_3163_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5285_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5285_0.pt f

/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1263_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1263_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1471_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1471_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1558_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1558_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5544_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5544_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_6204_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_6204_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1646_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1646_0.pt

...

/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_2521_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_2521_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4331_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4331_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_3300_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_3300_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_2753_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_2753_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5964_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5964_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5404_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5404_0.pt

/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1122_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1122_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_3364_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_3364_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4320_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4320_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_674_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_674_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4892_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4892_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_2941_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_2941_0.pt f

/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5953_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5953_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_426_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_426_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_6293_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_6293_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_3594_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_3594_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5324_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5324_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_3111_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_3111_0.pt f

/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4782_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4782_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1650_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1650_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5918_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5918_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1504_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_1504_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_2697_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_2697_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_3720_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_3720_0.pt

/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4303_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4303_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_2857_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_2857_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4576_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_4576_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5793_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5793_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_6422_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_6422_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5319_0.pt finish
/users/yewzzang/data_graph/pt/ttbar_weight_210719/alledge_w1/ttbar_5319_0.pt